In [7]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr  9 12:11:42 2020

Takes hypercube of layers and indexes and converts it to 

@author: adamt
"""

import os
import sys
import arcpy
import numpy as np
import pandas as pd
from arcpy.sa import *

#%%
def hlavni_strom(input_raster, output_name):

    ##  loading layers
    gdbPath = r"S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\machine_learning\Ryhy_Polygony_2.gdb"

    # To allow overwriting outputs change overwrite Output option to True.
    arcpy.env.overwriteOutput = True

    input_ras = arcpy.Raster(input_raster)
    
    ## cellsize specification
    cellSize = input_ras.meanCellWidth
    Cell_Size = str(cellSize)+" "+str(cellSize)

    ## additional values specification
    lowerLeft = arcpy.Point(input_ras.extent.XMin,input_ras.extent.YMin)

    ## export to numpy array
    input_array = arcpy.RasterToNumPyArray(input_ras,nodata_to_value=255)

    ## array dimensions
    dimensions = input_array.shape

    print("Preparation of compo array ", input_raster," is done")
    print(input_array.shape)

#     lim = 50

    height, width = input_array.shape[0], input_array.shape[1]

    # empty result arrays

    result_array = numpy.zeros((height, width))
    executed_array = numpy.zeros((height, width))
    process_array = numpy.zeros((height, width))

    # statistical data
    statistics = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    check = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    # seting the basic parameters
    max_x = width-1
    max_y = height-1
    # max_x = 1000
    # max_y = 1000
    ra = width * height

    # zatridovaci strom

    side = 0
    i,j = 0, 0
    a = 0
    c = 0
    percento = 0
    switch = 0
    C = 0
    print(max_x, max_y)
    for k in range(ra):
        statistics[0] += 1
        c += 1
        
        if input_array[j,i] >= 0.5 and input_array[j,i] <= 1:  
#             result_array[j,i] = input_array[j,i]
            result_array[j,i] = 1
            executed_array[j,i] = 1
            statistics[1] += 1
        else:
            pass

        if i == max_x:
            i = side
            j += 1
            a += 1
        else:
            i += 1

        if j == max_y:
            break
            
    side = 0
    i,j = 0, 0
    a = 0
    c = 0
    percento = 0
    switch = 0
    C = 0
    print(max_x, max_y)
    for k in range(ra):
        statistics[2] += 1
        c += 1
            
        if input_array[j,i] >= 0.5  and input_array[j,i] <= 1:
            statistics[3] += 1

            try:
                leva_horni_sum = result_array[j-1, i-1] + result_array[j-1, i] + result_array[j-1, i+1] + result_array[j, i-1] + result_array[j+1, i-1]
                statistics[4] += 1

                prava_horni_sum = result_array[j-1, i-1] + result_array[j-1, i] + result_array[j-1, i+1] + result_array[j, i+1] + result_array[j+1, i+1]
                statistics[5] += 1

                leva_dolni_sum = result_array[j-1, i-1] + result_array[j, i-1] + result_array[j+1, i-1] + result_array[j+1, i] + result_array[j+1, i+1]
                statistics[6] += 1

                prava_dolni_sum = result_array[j-1, i+1] + result_array[j, i+1] + result_array[j+1, i-1] + result_array[j+1, i] + result_array[j+1, i+1]
                statistics[7] += 1

                if leva_horni_sum == 0 or prava_horni_sum == 0 or leva_dolni_sum == 0 or prava_dolni_sum == 0:
                    result_array[j,i] = 0
                    executed_array[j,i] = 1
                    statistics[8] += 1
                else:
    #                     result_array[j,i] = input_array[j,i]
                    result_array[j,i] = 1
                    executed_array[j,i] = 1
                    statistics[9] += 1

            except:
                process_array[j,i] = 1
                statistics[10] += 1
                pass
                
        else:
            pass

        if i == max_x:
            i = side
            j += 1
            a += 1
        else:
            i += 1

        if j == max_y:
            break
            
    side = 0
    i,j = 0, 0
    a = 0
    c = 0
    percento = 0
    switch = 0
    C = 0
    print(max_x, max_y)
    for k in range(ra):
        statistics[11] += 1
        c += 1
            
        #   vse nad 75% i s okolim
        if input_array[j,i] >= 0.75  and input_array[j,i] <= 1:
            try:
#                 result_array[j-1:j+2, i-1:i+2] = input_array[j-1:j+2, i-1:i+2]
                result_array[j-1:j+2, i-1:i+2] = 1

#                 if C == 0:
#                     print(executed_array[j-3:j+3, i-3:i+3])

                executed_array[j-1:j+2, i-1:i+2] = 1
                statistics[12] += 1
#                 if C == 0:
# #                     print(executed_array[j-3:j+3, i-3:i+3])
#                     C = 1

            except:
                statistics[13] += 1
                process_array[j,i] = 1
                pass
        else:
            pass

        if i == max_x:
            i = side
            j += 1
            a += 1
        else:
            i += 1

        if j == max_y:
            break

#         percento_list = [10, 20, 30, 40, 50, 60, 70, 80, 90]

#         if round(c/ra *100, 0) in percento_list and switch == 0:
#             percento += 10
#             switch = 1
#             print(str(percento), "% ,", end = ' ')
#         elif round(c/ra *100, 0) not in percento_list:
#             switch = 0
#         else:
#             pass
        
    print(statistics)
    
    result_array = result_array.astype(int)

    resultRaster = arcpy.NumPyArrayToRaster(result_array, lowerLeft, cellSize, value_to_nodata=255)
#     processRaster = arcpy.NumPyArrayToRaster(process_array, lowerLeft, cellSize, value_to_nodata=255)

    resultRaster.save(output_name)
#     process_name = output_name.replace('filtered', 'processed')
#     processRaster.save(process_name)

#     projection = "PROJCS['S-JTSK_Krovak_East_North',GEOGCS['GCS_S_JTSK',DATUM['D_S_JTSK',SPHEROID['Bessel_1841',6377397.155,299.1528128]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Krovak'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Pseudo_Standard_Parallel_1',78.5],PARAMETER['Scale_Factor',0.9999],PARAMETER['Azimuth',30.28813975277778],PARAMETER['Longitude_Of_Center',24.83333333333333],PARAMETER['Latitude_Of_Center',49.5],PARAMETER['X_Scale',-1.0],PARAMETER['Y_Scale',1.0],PARAMETER['XY_Plane_Rotation',90.0],UNIT['Meter',1.0]]"

#     arcpy.management.DefineProjection(resultRaster, projection)

    print('Analysis of array ', input_raster," is done")


In [ ]:
# import libraries
import os
import sys
import arcpy
import numpy as np
import pandas as pd
from arcpy.sa import *
import csv

gdbPath = r"S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\machine_learning\STRIX_ML.gdb"
# input_folder = r"E:\DS_ML\model_001\result_rasters"
# output_folder = r"E:\DS_ML\model_001\filtered_rasters"

log_list = ['20181112_01', '20181112_02', '20181112_03', '20181115_02', '20181115_03', '20181123_02', '20181123_03', '20181124_01', '20181124_02', '20181127_01', '20181127_02', '20181127_03', '20181129_01', '20181129_02', '20181129_03', '20181203_01', '20181203_02', '20181203_03', '20181203_04', '20181206_01',
            '20181206_02', '20181206_03', '20181211_01', '20181211_02', '20181211_03', '20181211_04', '20181211_05', '20181211_06', '20181214_01', '20181214_02', '20181214_03', '20181219_01', '20181219_02', '20181219_03', '20190108_01', '20190108_02', '20190108_03', '20200708_01', '20200708_02', '20200716_01',
            '20200716_03', '20200721_01', '20200721_02', '20200721_03', '20200727_01', '20200727_02', '20200727_03', '20200729_01', '20200729_02', '20200729_03', '20200803_01', '20200803_03', '20200810_01', '20200824_01', '20200824_02', '20200827_02', '20200827_03', '20201102_01', '20201102_04',
            '20201102_05', '20201102_06', '20201105_01', '20201105_02', '20201105_03', '20201112_01', '20201112_02', '20201112_03', '20201112_04', '20201112_05', '20201119_01', '20201119_02', '20201119_03', '20201119_04', '20201119_05', '20201126_01', '20201126_02', '20201126_03', '20201126_04', '20201203_06']

# log_list = ['20181112_01']
error_list = []
model_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

typ_list = ['sheet', 'noRill', 'rill', ]

for model in model_list:
    model_name = "model_0" + model
    input_folder = os.path.join(r"D:\DS_ML\modely", model_name, "result_rasters")
    output_folder = os.path.join(r"D:\DS_ML\modely", model_name, "filtered_rasters")

    for item in log_list:
        name_list = item.split('_')
        for typ in typ_list:
    #         try:

            # get input name
            input_raster = os.path.join(input_folder, "List_ds_" + name_list[0] + "_" + name_list[1] + "_" + typ + "_avg.tif")

            # Save the output
            result_name = "List_ds_" + name_list[0] + "_" + name_list[1] + "_" + typ + "_filtered.tif"
            output_name = os.path.join(output_folder, result_name)

            hlavni_strom(input_raster, output_name)

    #         except:
    #             error_list.append(item, typ)

Preparation of compo array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20181112_01_sheet_avg.tif  is done
(159, 39)
38 158
38 158
38 158
[6162, 116, 6162, 116, 116, 116, 116, 116, 116, 0, 0, 6162, 59, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20181112_01_sheet_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20181112_01_noRill_avg.tif  is done
(159, 39)
38 158
38 158
38 158
[6162, 23, 6162, 23, 23, 23, 23, 23, 23, 0, 0, 6162, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20181112_01_noRill_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20181112_01_rill_avg.tif  is done
(159, 39)
38 158
38 158
38 158
[6162, 18, 6162, 18, 18, 18, 18, 18, 17, 1, 0, 6162, 10, 0, 0, 0, 0, 0, 0, 0, 0, 

[6162, 22, 6162, 22, 22, 22, 22, 22, 22, 0, 0, 6162, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20181129_03_sheet_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20181129_03_noRill_avg.tif  is done
(159, 39)
38 158
38 158
38 158
[6162, 128, 6162, 128, 128, 128, 128, 128, 122, 6, 0, 6162, 52, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20181129_03_noRill_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20181129_03_rill_avg.tif  is done
(159, 39)
38 158
38 158
38 158
[6162, 82, 6162, 82, 82, 82, 82, 82, 82, 0, 0, 6162, 36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20181129_03_rill_av

Preparation of compo array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20181214_01_noRill_avg.tif  is done
(159, 39)
38 158
38 158
38 158
[6162, 4, 6162, 4, 4, 4, 4, 4, 4, 0, 0, 6162, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20181214_01_noRill_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20181214_01_rill_avg.tif  is done
(159, 39)
38 158
38 158
38 158
[6162, 9, 6162, 9, 9, 9, 9, 9, 9, 0, 0, 6162, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20181214_01_rill_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20181214_02_sheet_avg.tif  is done
(159, 39)
38 158
38 158
38 158
[6162, 3, 6162, 3, 3, 3, 3, 3, 3, 0, 0, 6162, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Analysis of array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20200721_02_noRill_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20200721_02_rill_avg.tif  is done
(159, 39)
38 158
38 158
38 158
[6162, 92, 6162, 92, 92, 92, 92, 92, 92, 0, 0, 6162, 61, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20200721_02_rill_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20200721_03_sheet_avg.tif  is done
(159, 39)
38 158
38 158
38 158
[6162, 58, 6162, 58, 58, 58, 58, 58, 57, 1, 0, 6162, 38, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20200721_03_sheet_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20200721_03_noRill_avg.tif  is done
(159, 39)
38 158


Preparation of compo array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20200827_03_rill_avg.tif  is done
(159, 39)
38 158
38 158
38 158
[6162, 123, 6162, 123, 123, 123, 123, 123, 112, 11, 0, 6162, 58, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20200827_03_rill_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20201102_01_sheet_avg.tif  is done
(159, 39)
38 158
38 158
38 158
[6162, 1, 6162, 1, 1, 1, 1, 1, 1, 0, 0, 6162, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20201102_01_sheet_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_001\result_rasters\List_ds_20201102_01_noRill_avg.tif  is done
(159, 39)
38 158
38 158
38 158
[6162, 0, 6162, 0, 0, 0, 0, 0, 0, 0, 0, 6162, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

...

Preparation of compo array  D:\DS_ML\modely\model_005\result_rasters\List_ds_20190108_03_rill_avg.tif  is done
(106, 26)
25 105
25 105
25 105
[2730, 2, 2730, 2, 2, 2, 2, 2, 2, 0, 0, 2730, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_005\result_rasters\List_ds_20190108_03_rill_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_005\result_rasters\List_ds_20200708_01_sheet_avg.tif  is done
(106, 26)
25 105
25 105
25 105
[2730, 43, 2730, 43, 43, 43, 43, 43, 43, 0, 0, 2730, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_005\result_rasters\List_ds_20200708_01_sheet_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_005\result_rasters\List_ds_20200708_01_noRill_avg.tif  is done
(106, 26)
25 105
25 105
25 105
[2730, 35, 2730, 35, 34, 34, 34, 34, 34, 0, 1, 2730, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Analysis of array  D:\DS_ML\modely\model_005\result_rasters\List_ds_20200803_01_rill_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_005\result_rasters\List_ds_20200803_03_sheet_avg.tif  is done
(106, 26)
25 105
25 105
25 105
[2730, 42, 2730, 42, 41, 41, 41, 41, 41, 0, 1, 2730, 23, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_005\result_rasters\List_ds_20200803_03_sheet_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_005\result_rasters\List_ds_20200803_03_noRill_avg.tif  is done
(106, 26)
25 105
25 105
25 105
[2730, 73, 2730, 73, 70, 70, 70, 70, 70, 0, 3, 2730, 39, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_005\result_rasters\List_ds_20200803_03_noRill_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_005\result_rasters\List_ds_20200803_03_rill_avg.tif  is done
(106, 26)
25 105


[2730, 40, 2730, 40, 40, 40, 40, 40, 39, 1, 0, 2730, 25, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_005\result_rasters\List_ds_20201112_02_sheet_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_005\result_rasters\List_ds_20201112_02_noRill_avg.tif  is done
(106, 26)
25 105
25 105
25 105
[2730, 41, 2730, 41, 40, 40, 40, 40, 35, 5, 1, 2730, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_005\result_rasters\List_ds_20201112_02_noRill_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_005\result_rasters\List_ds_20201112_02_rill_avg.tif  is done
(106, 26)
25 105
25 105
25 105
[2730, 64, 2730, 64, 62, 62, 62, 62, 60, 2, 2, 2730, 41, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_005\result_rasters\List_ds_20201112_02_rill_avg.tif 

Preparation of compo array  D:\DS_ML\modely\model_006\result_rasters\List_ds_20181112_01_noRill_avg.tif  is done
(106, 26)
25 105
25 105
25 105
[2730, 2, 2730, 2, 2, 2, 2, 2, 2, 0, 0, 2730, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_006\result_rasters\List_ds_20181112_01_noRill_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_006\result_rasters\List_ds_20181112_01_rill_avg.tif  is done
(106, 26)
25 105
25 105
25 105
[2730, 3, 2730, 3, 3, 3, 3, 3, 3, 0, 0, 2730, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_006\result_rasters\List_ds_20181112_01_rill_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_006\result_rasters\List_ds_20181112_02_sheet_avg.tif  is done
(106, 26)
25 105
25 105
25 105
[2730, 55, 2730, 55, 55, 55, 55, 55, 55, 0, 0, 2730, 31, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Analysis of array  D:\DS_ML\modely\model_006\result_rasters\List_ds_20181129_03_noRill_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_006\result_rasters\List_ds_20181129_03_rill_avg.tif  is done
(106, 26)
25 105
25 105
25 105
[2730, 40, 2730, 40, 40, 40, 40, 40, 40, 0, 0, 2730, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_006\result_rasters\List_ds_20181129_03_rill_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_006\result_rasters\List_ds_20181203_01_sheet_avg.tif  is done
(106, 26)
25 105
25 105
25 105
[2730, 15, 2730, 15, 11, 11, 11, 11, 11, 0, 4, 2730, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  D:\DS_ML\modely\model_006\result_rasters\List_ds_20181203_01_sheet_avg.tif  is done
Preparation of compo array  D:\DS_ML\modely\model_006\result_rasters\List_ds_20181203_01_noRill_avg.tif  is done
(106, 26)
25 105


In [7]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jan  2 12:11:42 2022

From filtered rasters create final judged raster comprising of three classes

@author: adamt
"""

import os
import sys
import arcpy
import numpy as np
import pandas as pd
import csv
from arcpy.sa import *

#%%
def judge_filtered_matrix(input_NoRill_raster, input_Sheet_raster, input_Rill_raster, output_name):

    ##  loading layers
    gdbPath = r"S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\machine_learning\STRIX_ML.gdb"

    # To allow overwriting outputs change overwrite Output option to True.
    arcpy.env.overwriteOutput = True

    input_NoRill_ras = arcpy.Raster(input_NoRill_raster)
    input_Sheet_ras = arcpy.Raster(input_Sheet_raster)
    input_Rill_ras = arcpy.Raster(input_Rill_raster)
    
    ## cellsize specification
    cellSize = input_NoRill_ras.meanCellWidth
    Cell_Size = str(cellSize)+" "+str(cellSize)

    ## additional values specification
    lowerLeft = arcpy.Point(input_NoRill_ras.extent.XMin,input_NoRill_ras.extent.YMin)

    ## export to numpy array
    input_NoRill_array = arcpy.RasterToNumPyArray(input_NoRill_ras,nodata_to_value=255)
    input_Sheet_array = arcpy.RasterToNumPyArray(input_Sheet_ras,nodata_to_value=255)
    input_Rill_array = arcpy.RasterToNumPyArray(input_Rill_ras,nodata_to_value=255)

    ## array dimensions
    dimensions = input_NoRill_array.shape
    height, width = dimensions[0], dimensions[1]
    
    print(input_NoRill_array.shape, input_Sheet_array.shape, input_Rill_array.shape)

    print("Preparation of compo array ", input_NoRill_raster," is done")
    print(input_NoRill_array.shape)
        
    result_array = numpy.zeros((height, width))
    executed_array = numpy.zeros((height, width))
    
    # statistical data
    statistics = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    check = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    # seting the basic parameters
    max_x = width-1
    max_y = height-1
    # max_x = 1000
    # max_y = 1000
    ra = width * height
    
    NoRill, Sheet, Rill = 0, 1, 2

    # zatridovaci strom

    side = 0
    i,j = 0, 0
    a = 0
    c = 0
    percento = 0
    switch = 0
    C = 0
    print(max_x, max_y)
    for k in range(ra):
        statistics[0] += 1
        c += 1
        
        if input_NoRill_array[j,i] == 1:  
            result_array[j,i] = 0
            executed_array[j,i] = 1
            statistics[1] += 1
        elif input_Sheet_array[j,i] == 1:  
            result_array[j,i] = 1
            executed_array[j,i] = 1
            statistics[2] += 1
        elif input_Rill_array[j,i] == 1:  
            result_array[j,i] = 2
            executed_array[j,i] = 1
            statistics[3] += 1
        else:
            statistics[4] += 1

        if i == max_x:
            i = side
            j += 1
            a += 1
        else:
            i += 1

        if j == max_y:
            break
            
        percento_list = [10, 20, 30, 40, 50, 60, 70, 80, 90]

        if round(c/ra *100, 0) in percento_list and switch == 0:
            percento += 10
            switch = 1
            print(str(percento), "% ,", end = ' ')
        elif round(c/ra *100, 0) not in percento_list:
            switch = 0
        else:
            pass
    
    print(statistics)
    
    result_array = result_array.astype(int)

    resultRaster = arcpy.NumPyArrayToRaster(result_array, lowerLeft, cellSize, value_to_nodata=255)
#     processRaster = arcpy.NumPyArrayToRaster(process_array, lowerLeft, cellSize, value_to_nodata=255)

    resultRaster.save(output_name)
#     process_name = output_name.replace('filtered', 'processed')
#     processRaster.save(process_name)

#     projection = "PROJCS['S-JTSK_Krovak_East_North',GEOGCS['GCS_S_JTSK',DATUM['D_S_JTSK',SPHEROID['Bessel_1841',6377397.155,299.1528128]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Krovak'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Pseudo_Standard_Parallel_1',78.5],PARAMETER['Scale_Factor',0.9999],PARAMETER['Azimuth',30.28813975277778],PARAMETER['Longitude_Of_Center',24.83333333333333],PARAMETER['Latitude_Of_Center',49.5],PARAMETER['X_Scale',-1.0],PARAMETER['Y_Scale',1.0],PARAMETER['XY_Plane_Rotation',90.0],UNIT['Meter',1.0]]"

#     arcpy.management.DefineProjection(resultRaster, projection)

    print('Analysis of array ', input_NoRill_raster," is done")
    

In [14]:
import os

storage_location = r"E:\DS_ML\model_001"
log_list = ['20181112_01', '20181112_02', '20181112_03', '20181115_02', '20181115_03', '20181123_02', '20181123_03', '20181124_01', '20181124_02', '20181127_01', '20181127_02', '20181127_03', '20181129_01', '20181129_02', '20181129_03', '20181203_01', '20181203_02', '20181203_03', '20181203_04', '20181206_01',
            '20181206_02', '20181206_03', '20181211_01', '20181211_02', '20181211_03', '20181211_04', '20181211_05', '20181211_06', '20181214_01', '20181214_02', '20181214_03', '20181219_01', '20181219_02', '20181219_03', '20190108_01', '20190108_02', '20190108_03', '20200708_01', '20200708_02', '20200716_01',
            '20200716_03', '20200721_01', '20200721_02', '20200721_03', '20200727_01', '20200727_02', '20200727_03', '20200729_01', '20200729_02', '20200729_03', '20200803_01', '20200803_03', '20200810_01', '20200824_01', '20200824_02', '20200827_02', '20200827_03', '20201102_01', '20201102_04',
            '20201102_05', '20201102_06', '20201105_01', '20201105_02', '20201105_03', '20201112_01', '20201112_02', '20201112_03', '20201112_04', '20201112_05', '20201119_01', '20201119_02', '20201119_03', '20201119_04', '20201119_05', '20201126_01', '20201126_02', '20201126_03', '20201126_04', '20201203_06']

# log_list = ['20181112_01']
for model in model_list:
    model_name = "model_0" + model
    storage_location = os.path.join(r"D:\DS_ML\modely", model_name)

    for item in log_list:
    #     loc = item.split('_')
    #     loc = loc[0] + "_" + loc[1]
        loc = item
        print(item, loc)

        input_NoRill_raster = os.path.join(storage_location, "filtered_rasters", "List_ds_" + loc + "_noRill_filtered.tif")
        input_Sheet_raster = os.path.join(storage_location, "filtered_rasters", "List_ds_" + loc + "_sheet_filtered.tif")
        input_Rill_raster = os.path.join(storage_location, "filtered_rasters", "List_ds_" + loc + "_rill_filtered.tif")

        output_name = os.path.join(storage_location, "judged_rasters", "List_ds_" + loc + "_judged.tif")

        judge_filtered_matrix(input_NoRill_raster, input_Sheet_raster, input_Rill_raster, output_name)

print('done')

20181112_01 20181112_01
(159, 39) (159, 39) (159, 39)
Preparation of compo array  E:\DS_ML\model_001\filtered_rasters\List_ds_20181112_01_noRill_filtered.tif  is done
(159, 39)
38 158
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , [6162, 32, 448, 60, 5622, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  E:\DS_ML\model_001\filtered_rasters\List_ds_20181112_01_noRill_filtered.tif  is done
20181112_02 20181112_02
(159, 39) (159, 39) (159, 39)
Preparation of compo array  E:\DS_ML\model_001\filtered_rasters\List_ds_20181112_02_noRill_filtered.tif  is done
(159, 39)
38 158
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , [6162, 124, 396, 102, 5540, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  E:\DS_ML\model_001\filtered_rasters\List_ds_20181112_02_noRill_filtered.tif  is done
20181112_03 20181112_03
(159, 39) 

Analysis of array  E:\DS_ML\model_001\filtered_rasters\List_ds_20181219_03_noRill_filtered.tif  is done
20190108_01 20190108_01
(158, 39) (158, 39) (158, 39)
Preparation of compo array  E:\DS_ML\model_001\filtered_rasters\List_ds_20190108_01_noRill_filtered.tif  is done
(158, 39)
38 157
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , [6123, 254, 148, 426, 5295, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  E:\DS_ML\model_001\filtered_rasters\List_ds_20190108_01_noRill_filtered.tif  is done
20190108_02 20190108_02
(159, 39) (159, 39) (159, 39)
Preparation of compo array  E:\DS_ML\model_001\filtered_rasters\List_ds_20190108_02_noRill_filtered.tif  is done
(159, 39)
38 158
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , [6162, 136, 16, 168, 5842, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  E:\DS_ML\model

38 158
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , [6162, 236, 60, 236, 5630, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  E:\DS_ML\model_001\filtered_rasters\List_ds_20201112_05_noRill_filtered.tif  is done
20201119_01 20201119_01
(159, 39) (159, 39) (159, 39)
Preparation of compo array  E:\DS_ML\model_001\filtered_rasters\List_ds_20201119_01_noRill_filtered.tif  is done
(159, 39)
38 158
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , [6162, 28, 52, 0, 6082, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Analysis of array  E:\DS_ML\model_001\filtered_rasters\List_ds_20201119_01_noRill_filtered.tif  is done
20201119_02 20201119_02
(159, 39) (159, 39) (159, 39)
Preparation of compo array  E:\DS_ML\model_001\filtered_rasters\List_ds_20201119_02_noRill_filtered.tif  is done
(159, 39)
38 158
10 % , 20 % , 30 % , 40 % , 50

In [1]:
import arcpy
import os

log_list = ['20181112_01', '20181112_02', '20181112_03', '20181115_02', '20181115_03', '20181123_02', '20181123_03', '20181124_01', '20181124_02', '20181127_01', '20181127_02', '20181127_03', '20181129_01', '20181129_02', '20181129_03', '20181203_01', '20181203_02', '20181203_03', '20181203_04', '20181206_01',
            '20181206_02', '20181206_03', '20181211_01', '20181211_02', '20181211_03', '20181211_04', '20181211_05', '20181211_06', '20181214_01', '20181214_02', '20181214_03', '20181219_01', '20181219_02', '20181219_03', '20190108_01', '20190108_02', '20190108_03', '20200708_01', '20200708_02', '20200716_01',
            '20200716_03', '20200721_01', '20200721_02', '20200721_03', '20200727_01', '20200727_02', '20200727_03', '20200729_01', '20200729_02', '20200729_03', '20200803_01', '20200803_03', '20200810_01', '20200824_01', '20200824_02', '20200827_02', '20200827_03', '20201102_01', '20201102_04',
            '20201102_05', '20201102_06', '20201105_01', '20201105_02', '20201105_03', '20201112_01', '20201112_02', '20201112_03', '20201112_04', '20201112_05', '20201119_01', '20201119_02', '20201119_03', '20201119_04', '20201119_05', '20201126_01', '20201126_02', '20201126_03', '20201126_04', '20201203_06']

# storage_location = r"E:\DS_ML\model_001"

model_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

for model in model_list:
    model_name = "model_0" + model
    storage_location = os.path.join(r"D:\DS_ML\modely", model_name)

    for item in log_list:
    #     loc = item.split("_")
    #     print(loc)
    #     loc = loc[2] + "_" + loc[3]
        loc = item
        print(loc)
        # To allow overwriting outputs change overwriteOutput option to True.
        arcpy.env.overwriteOutput = True

        validation_raster = "Rastery\\Raster_" + loc
        classified_raster = os.path.join(storage_location, "judged_rasters", "List_ds_" + loc + "_judged.tif")

        validation = arcpy.Raster(validation_raster)
        classified = arcpy.Raster(classified_raster)
        print(validation_raster, classified_raster)

        # Process: Composite Bands (Composite Bands) (management)
        output = os.path.join(storage_location, "validation.gdb\\" + "List_ds_" + loc + "_validation")

        arcpy.management.CompositeBands(in_rasters=[validation, classified], out_raster= output)
    #     output = arcpy.Raster(output)


20181112_01
Rastery\Raster_20181112_01 D:\DS_ML\modely\model_001\judged_rasters\List_ds_20181112_01_judged.tif
20181112_02
Rastery\Raster_20181112_02 D:\DS_ML\modely\model_001\judged_rasters\List_ds_20181112_02_judged.tif
20181112_03
Rastery\Raster_20181112_03 D:\DS_ML\modely\model_001\judged_rasters\List_ds_20181112_03_judged.tif
20181115_02
Rastery\Raster_20181115_02 D:\DS_ML\modely\model_001\judged_rasters\List_ds_20181115_02_judged.tif
20181115_03
Rastery\Raster_20181115_03 D:\DS_ML\modely\model_001\judged_rasters\List_ds_20181115_03_judged.tif
20181123_02
Rastery\Raster_20181123_02 D:\DS_ML\modely\model_001\judged_rasters\List_ds_20181123_02_judged.tif
20181123_03
Rastery\Raster_20181123_03 D:\DS_ML\modely\model_001\judged_rasters\List_ds_20181123_03_judged.tif
20181124_01
Rastery\Raster_20181124_01 D:\DS_ML\modely\model_001\judged_rasters\List_ds_20181124_01_judged.tif
20181124_02
Rastery\Raster_20181124_02 D:\DS_ML\modely\model_001\judged_rasters\List_ds_20181124_02_judged.tif
2

Rastery\Raster_20201112_04 D:\DS_ML\modely\model_002\judged_rasters\List_ds_20201112_04_judged.tif
20201112_05
Rastery\Raster_20201112_05 D:\DS_ML\modely\model_002\judged_rasters\List_ds_20201112_05_judged.tif
20201119_01
Rastery\Raster_20201119_01 D:\DS_ML\modely\model_002\judged_rasters\List_ds_20201119_01_judged.tif
20201119_02
Rastery\Raster_20201119_02 D:\DS_ML\modely\model_002\judged_rasters\List_ds_20201119_02_judged.tif
20201119_03
Rastery\Raster_20201119_03 D:\DS_ML\modely\model_002\judged_rasters\List_ds_20201119_03_judged.tif
20201119_04
Rastery\Raster_20201119_04 D:\DS_ML\modely\model_002\judged_rasters\List_ds_20201119_04_judged.tif
20201119_05
Rastery\Raster_20201119_05 D:\DS_ML\modely\model_002\judged_rasters\List_ds_20201119_05_judged.tif
20201126_01
Rastery\Raster_20201126_01 D:\DS_ML\modely\model_002\judged_rasters\List_ds_20201126_01_judged.tif
20201126_02
Rastery\Raster_20201126_02 D:\DS_ML\modely\model_002\judged_rasters\List_ds_20201126_02_judged.tif
20201126_03
R

Rastery\Raster_20200827_02 D:\DS_ML\modely\model_004\judged_rasters\List_ds_20200827_02_judged.tif
20200827_03
Rastery\Raster_20200827_03 D:\DS_ML\modely\model_004\judged_rasters\List_ds_20200827_03_judged.tif
20201102_01
Rastery\Raster_20201102_01 D:\DS_ML\modely\model_004\judged_rasters\List_ds_20201102_01_judged.tif
20201102_04
Rastery\Raster_20201102_04 D:\DS_ML\modely\model_004\judged_rasters\List_ds_20201102_04_judged.tif
20201102_05
Rastery\Raster_20201102_05 D:\DS_ML\modely\model_004\judged_rasters\List_ds_20201102_05_judged.tif
20201102_06
Rastery\Raster_20201102_06 D:\DS_ML\modely\model_004\judged_rasters\List_ds_20201102_06_judged.tif
20201105_01
Rastery\Raster_20201105_01 D:\DS_ML\modely\model_004\judged_rasters\List_ds_20201105_01_judged.tif
20201105_02
Rastery\Raster_20201105_02 D:\DS_ML\modely\model_004\judged_rasters\List_ds_20201105_02_judged.tif
20201105_03
Rastery\Raster_20201105_03 D:\DS_ML\modely\model_004\judged_rasters\List_ds_20201105_03_judged.tif
20201112_01
R

Rastery\Raster_20200721_03 D:\DS_ML\modely\model_006\judged_rasters\List_ds_20200721_03_judged.tif
20200727_01
Rastery\Raster_20200727_01 D:\DS_ML\modely\model_006\judged_rasters\List_ds_20200727_01_judged.tif
20200727_02
Rastery\Raster_20200727_02 D:\DS_ML\modely\model_006\judged_rasters\List_ds_20200727_02_judged.tif
20200727_03
Rastery\Raster_20200727_03 D:\DS_ML\modely\model_006\judged_rasters\List_ds_20200727_03_judged.tif
20200729_01
Rastery\Raster_20200729_01 D:\DS_ML\modely\model_006\judged_rasters\List_ds_20200729_01_judged.tif
20200729_02
Rastery\Raster_20200729_02 D:\DS_ML\modely\model_006\judged_rasters\List_ds_20200729_02_judged.tif
20200729_03
Rastery\Raster_20200729_03 D:\DS_ML\modely\model_006\judged_rasters\List_ds_20200729_03_judged.tif
20200803_01
Rastery\Raster_20200803_01 D:\DS_ML\modely\model_006\judged_rasters\List_ds_20200803_01_judged.tif
20200803_03
Rastery\Raster_20200803_03 D:\DS_ML\modely\model_006\judged_rasters\List_ds_20200803_03_judged.tif
20200810_01
R

Rastery\Raster_20181219_01 D:\DS_ML\modely\model_008\judged_rasters\List_ds_20181219_01_judged.tif
20181219_02
Rastery\Raster_20181219_02 D:\DS_ML\modely\model_008\judged_rasters\List_ds_20181219_02_judged.tif
20181219_03
Rastery\Raster_20181219_03 D:\DS_ML\modely\model_008\judged_rasters\List_ds_20181219_03_judged.tif
20190108_01
Rastery\Raster_20190108_01 D:\DS_ML\modely\model_008\judged_rasters\List_ds_20190108_01_judged.tif
20190108_02
Rastery\Raster_20190108_02 D:\DS_ML\modely\model_008\judged_rasters\List_ds_20190108_02_judged.tif
20190108_03
Rastery\Raster_20190108_03 D:\DS_ML\modely\model_008\judged_rasters\List_ds_20190108_03_judged.tif
20200708_01
Rastery\Raster_20200708_01 D:\DS_ML\modely\model_008\judged_rasters\List_ds_20200708_01_judged.tif
20200708_02
Rastery\Raster_20200708_02 D:\DS_ML\modely\model_008\judged_rasters\List_ds_20200708_02_judged.tif
20200716_01
Rastery\Raster_20200716_01 D:\DS_ML\modely\model_008\judged_rasters\List_ds_20200716_01_judged.tif
20200716_03
R

Rastery\Raster_20181206_01 D:\DS_ML\modely\model_010\judged_rasters\List_ds_20181206_01_judged.tif
20181206_02
Rastery\Raster_20181206_02 D:\DS_ML\modely\model_010\judged_rasters\List_ds_20181206_02_judged.tif
20181206_03
Rastery\Raster_20181206_03 D:\DS_ML\modely\model_010\judged_rasters\List_ds_20181206_03_judged.tif
20181211_01
Rastery\Raster_20181211_01 D:\DS_ML\modely\model_010\judged_rasters\List_ds_20181211_01_judged.tif
20181211_02
Rastery\Raster_20181211_02 D:\DS_ML\modely\model_010\judged_rasters\List_ds_20181211_02_judged.tif
20181211_03
Rastery\Raster_20181211_03 D:\DS_ML\modely\model_010\judged_rasters\List_ds_20181211_03_judged.tif
20181211_04
Rastery\Raster_20181211_04 D:\DS_ML\modely\model_010\judged_rasters\List_ds_20181211_04_judged.tif
20181211_05
Rastery\Raster_20181211_05 D:\DS_ML\modely\model_010\judged_rasters\List_ds_20181211_05_judged.tif
20181211_06
Rastery\Raster_20181211_06 D:\DS_ML\modely\model_010\judged_rasters\List_ds_20181211_06_judged.tif
20181214_01
R

Rastery\Raster_20181124_01 D:\DS_ML\modely\model_012\judged_rasters\List_ds_20181124_01_judged.tif
20181124_02
Rastery\Raster_20181124_02 D:\DS_ML\modely\model_012\judged_rasters\List_ds_20181124_02_judged.tif
20181127_01
Rastery\Raster_20181127_01 D:\DS_ML\modely\model_012\judged_rasters\List_ds_20181127_01_judged.tif
20181127_02
Rastery\Raster_20181127_02 D:\DS_ML\modely\model_012\judged_rasters\List_ds_20181127_02_judged.tif
20181127_03
Rastery\Raster_20181127_03 D:\DS_ML\modely\model_012\judged_rasters\List_ds_20181127_03_judged.tif
20181129_01
Rastery\Raster_20181129_01 D:\DS_ML\modely\model_012\judged_rasters\List_ds_20181129_01_judged.tif
20181129_02
Rastery\Raster_20181129_02 D:\DS_ML\modely\model_012\judged_rasters\List_ds_20181129_02_judged.tif
20181129_03
Rastery\Raster_20181129_03 D:\DS_ML\modely\model_012\judged_rasters\List_ds_20181129_03_judged.tif
20181203_01
Rastery\Raster_20181203_01 D:\DS_ML\modely\model_012\judged_rasters\List_ds_20181203_01_judged.tif
20181203_02
R

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr  9 12:11:42 2020

Takes hypercube of layers and indexes and converts it to 

@author: adamt
"""

import os
import sys
import arcpy
import numpy as np
import pandas as pd
import csv
from arcpy.sa import *

# input_raster = "Validation_31//Validation_Bykovice1_20210510" 
# loc = "Bykovice1_20210510"
# storage_location = r"Z:\DPZ_drony\Modely\model_003\mosaics"

#%%
def validation_matrix(input_raster, loc, storage_location):

    ##  loading layers
    gdbPath = r"Z:\DPZ_drony\DPZ_drony.gdb"

    # To allow overwriting outputs change overwrite Output option to True.
    arcpy.env.overwriteOutput = True

    input_ras = arcpy.Raster(input_raster)

    ## cellsize specification
    cellSize = input_ras.meanCellWidth
    Cell_Size = str(cellSize)+" "+str(cellSize)

    ## additional values specification
    lowerLeft = arcpy.Point(input_ras.extent.XMin,input_ras.extent.YMin)

    ## export to numpy array
    input_array = arcpy.RasterToNumPyArray(input_ras,nodata_to_value=255)

    ## array dimensions
    dimensions = input_array.shape

    print("Preparation of compo array ", input_raster," is done")
    print(input_array.shape)

    height, width = input_array.shape[1], input_array.shape[2]
    ra = width * height
    print(ra)

    # seting the basic parameters
    max_x = width-1
    max_y = height-1

    side = 0
    i,j = 0, 0
    a = 0
    A = 0
    c = 0
    percento = 0
    switch = 0
    C = 0

    validation_array = 0
    classified_array = 1

    validation_matrix = np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]])

    for k in range(ra):
        c += 1

        if input_array[validation_array][j,i] >= 900 and input_array[validation_array][j,i] <= 999 and input_array[classified_array][j,i] == 0:  
            validation_matrix[0,0] += 1

        elif input_array[validation_array][j,i] >= 800 and input_array[validation_array][j,i] <= 899 and input_array[classified_array][j,i] == 0:  
            validation_matrix[0,1] += 1

        elif input_array[validation_array][j,i] >= 700 and input_array[validation_array][j,i] <= 799 and input_array[classified_array][j,i] == 0:  
            validation_matrix[0,2] += 1

        elif input_array[validation_array][j,i] >= 900 and input_array[validation_array][j,i] <= 999 and input_array[classified_array][j,i] == 1:  
            validation_matrix[1,0] += 1

        elif input_array[validation_array][j,i] >= 800 and input_array[validation_array][j,i] <= 899 and input_array[classified_array][j,i] == 1:  
            validation_matrix[1,1] += 1

        elif input_array[validation_array][j,i] >= 700 and input_array[validation_array][j,i] <= 799 and input_array[classified_array][j,i] == 1:  
            validation_matrix[1,2] += 1

        elif input_array[validation_array][j,i] >= 900 and input_array[validation_array][j,i] <= 999 and input_array[classified_array][j,i] == 2:  
            validation_matrix[2,0] += 1

        elif input_array[validation_array][j,i] >= 800 and input_array[validation_array][j,i] <= 899 and input_array[classified_array][j,i] == 2:  
            validation_matrix[2,1] += 1

        elif input_array[validation_array][j,i] >= 700 and input_array[validation_array][j,i] <= 799 and input_array[classified_array][j,i] == 2:  
            validation_matrix[2,2] += 1

        else:
            pass

        if i == max_x:
            i = side
            j += 1
            a += 1
        else:
            i += 1

        if j == max_y:
            break

        percento_list = [10, 20, 30, 40, 50, 60, 70, 80, 90]

        if round(A/ra *100, 0) in percento_list and switch == 0:
            percento += 10
            switch = 1
            print(str(percento), "% ,", end = ' ')
        elif round(A/ra *100, 0) not in percento_list:
            switch = 0
        else:
            pass

        A += 1

    print()

#     print(validation_matrix)

    total_valid_NoRill = validation_matrix[0,0] + validation_matrix[1,0] + validation_matrix[2,0]
    total_valid_Sheet = validation_matrix[0,1] + validation_matrix[1,1] + validation_matrix[2,1]
    total_valid_Rill = validation_matrix[0,2] + validation_matrix[1,2] + validation_matrix[2,2]
    
    validation_matrix_pct = np.array([[validation_matrix[0,0]/total_valid_NoRill*100, validation_matrix[0,1]/total_valid_Sheet*100, validation_matrix[0,2]/total_valid_Rill*100],
                                  [validation_matrix[1,0]/total_valid_NoRill*100, validation_matrix[1,1]/total_valid_Sheet*100, validation_matrix[1,2]/total_valid_Rill*100],
                                  [validation_matrix[2,0]/total_valid_NoRill*100, validation_matrix[2,1]/total_valid_Sheet*100, validation_matrix[2,2]/total_valid_Rill*100]])

#     print(validation_matrix_pct)
    
    total_class_NoRill = validation_matrix[0,0] + validation_matrix[0,1] + validation_matrix[0,2]
    total_class_Sheet = validation_matrix[1,0] + validation_matrix[1,1] + validation_matrix[1,2]
    total_class_Rill = validation_matrix[2,0] + validation_matrix[2,1] + validation_matrix[2,2]
    
    classification_matrix_pct = np.array([[validation_matrix[0,0]/total_class_NoRill*100, validation_matrix[0,1]/total_class_NoRill*100, validation_matrix[0,2]/total_class_NoRill*100],
                                      [validation_matrix[1,0]/total_class_Sheet*100, validation_matrix[1,1]/total_class_Sheet*100, validation_matrix[1,2]/total_class_Sheet*100],
                                      [validation_matrix[2,0]/total_class_Rill*100, validation_matrix[2,1]/total_class_Rill*100, validation_matrix[2,2]/total_class_Rill*100]])
    
#     print(classification_matrix_pct)

    # print validation matrix
    printing_block = [["",         "",                     loc,                                 ""                          ],
                      ["",         "",                     "",                  "Validation Matrix"                         ],
                      ["",                                                   "Validation Raster"                            ],
                      ["",         "",                   "NoRill",                "Sheet",                      "Rill"      ],
                      ["Classified Raster", "NoRill", round(validation_matrix_pct[0,0],0), round(validation_matrix_pct[0,1],0), round(validation_matrix_pct[0,2],0)],
                      ["",                   "Sheet", round(validation_matrix_pct[1,0],0), round(validation_matrix_pct[1,1],0), round(validation_matrix_pct[1,2],0)],
                      ["",                    "Rill", round(validation_matrix_pct[2,0],0), round(validation_matrix_pct[2,1],0), round(validation_matrix_pct[2,2],0)],
                      [""],
                      ["",                                                   "Clasification Raster"                         ],
                      ["",         "",                   "NoRill",                "Sheet",                      "Rill"      ],
                      ["Classified Raster", "NoRill", round(classification_matrix_pct[0,0],0), round(classification_matrix_pct[0,1],0), round(classification_matrix_pct[0,2],0)],
                      ["",                   "Sheet", round(classification_matrix_pct[1,0],0), round(classification_matrix_pct[1,1],0), round(classification_matrix_pct[1,2],0)],
                      ["",                    "Rill", round(classification_matrix_pct[2,0],0), round(classification_matrix_pct[2,1],0), round(classification_matrix_pct[2,2],0)],]

    ## creation of output file name
    fileName = "List_" + loc + "_validation.csv"
    outputFileName = os.path.join(storage_location, fileName)

    with open(outputFileName, mode='w', encoding = "utf-8", newline='') as output_csv:
        csv_writer = csv.writer(output_csv, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for line in printing_block:
            csv_writer.writerow(line)
            
    stat_line = [loc,
                 validation_matrix_pct[0,0], validation_matrix_pct[0,1], validation_matrix_pct[0,2],
                 validation_matrix_pct[1,0], validation_matrix_pct[1,1], validation_matrix_pct[1,2],
                 validation_matrix_pct[2,0], validation_matrix_pct[2,1], validation_matrix_pct[2,2],
                 "",
                 classification_matrix_pct[0,0], classification_matrix_pct[0,1], classification_matrix_pct[0,2],
                 classification_matrix_pct[1,0], classification_matrix_pct[1,1], classification_matrix_pct[1,2],
                 classification_matrix_pct[2,0], classification_matrix_pct[2,1], classification_matrix_pct[2,2]]
                 
#     print()
    print("Validation matrix of ", loc, ' done')
    print()
    
    return stat_line

In [3]:
# input_raster = "Validation_31//Validation_Bykovice1_20210510" 
# loc = "Bykovice1_20210510"

# storage_location = r"E:\DS_ML\model_001"

log_list = ['20181112_01', '20181112_02', '20181112_03', '20181115_02', '20181115_03', '20181123_02', '20181123_03', '20181124_01', '20181124_02', '20181127_01', '20181127_02', '20181127_03', '20181129_01', '20181129_02', '20181129_03', '20181203_01', '20181203_02', '20181203_03', '20181203_04', '20181206_01',
            '20181206_02', '20181206_03', '20181211_01', '20181211_02', '20181211_03', '20181211_04', '20181211_05', '20181211_06', '20181214_01', '20181214_02', '20181214_03', '20181219_01', '20181219_02', '20181219_03', '20190108_01', '20190108_02', '20190108_03', '20200708_01', '20200708_02', '20200716_01',
            '20200716_03', '20200721_01', '20200721_02', '20200721_03', '20200727_01', '20200727_02', '20200727_03', '20200729_01', '20200729_02', '20200729_03', '20200803_01', '20200803_03', '20200810_01', '20200824_01', '20200824_02', '20200827_02', '20200827_03', '20201102_01', '20201102_04',
            '20201102_05', '20201102_06', '20201105_01', '20201105_02', '20201105_03', '20201112_01', '20201112_02', '20201112_03', '20201112_04', '20201112_05', '20201119_01', '20201119_02', '20201119_03', '20201119_04', '20201119_05', '20201126_01', '20201126_02', '20201126_03', '20201126_04', '20201203_06']

# storage_location = r"E:\DS_ML\model_001"

# model_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
model_list = ['04', '05', '06', '07', '08', '09', '10', '11', '12']

for model in model_list:
    model_name = "model_0" + model
    storage_location = os.path.join(r"D:\DS_ML\modely", model_name)
    
    stat_list = [["", "", "", "", "", "validation_matrix_pct", "", "", "", "",
              "",
              "", "", "", "", "", "classification_matrix_pct", "", "", "", ""],
             ["",
              "NoRill/NoRill", "Sheet/NoRill", "Rill/NoRill", "NoRill/Sheet", "Sheet/Sheet", "Rill/Sheet", "NoRill/Rill", "Sheet/Rill", "Rill/Rill",
              "",
              "NoRill/NoRill", "Sheet/NoRill", "Rill/NoRill", "NoRill/Sheet", "Sheet/Sheet", "Rill/Sheet", "NoRill/Rill", "Sheet/Rill", "Rill/Rill",]]


    for item in log_list:
    #     loc = item.split('_')
    #     loc = loc[2] + "_" + loc[3]
        loc = item
        print(item, loc, storage_location)
#         input_composite = "Results//Model_001_validation\\List_ds_" + loc + "_001_validation"
        input_composite = os.path.join(storage_location, "validation.gdb\\" + "List_ds_" + loc + "_001_validation")
        stat_line = validation_matrix(input_composite, loc, storage_location)

        stat_list.append(stat_line)

    fileName = "Validation_model_0" + model + ".csv"
    outputFileName = os.path.join(storage_location, fileName)

    with open(outputFileName, mode='w', encoding = "utf-8", newline='') as output_csv:
        csv_writer = csv.writer(output_csv, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for line in stat_list:
            csv_writer.writerow(line)

print('done')


20181112_01 20181112_01 D:\DS_ML\modely\model_004
Preparation of compo array  D:\DS_ML\modely\model_004\validation.gdb\List_ds_20181112_01_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20181112_01  done

20181112_02 20181112_02 D:\DS_ML\modely\model_004
Preparation of compo array  D:\DS_ML\modely\model_004\validation.gdb\List_ds_20181112_02_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20181112_02  done

20181112_03 20181112_03 D:\DS_ML\modely\model_004
Preparation of compo array  D:\DS_ML\modely\model_004\validation.gdb\List_ds_20181112_03_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20181112_03  done

20181115_02 20181115_02 D:\DS_ML\modely\model_004
Preparation of compo array  D:\DS_ML\modely\model_004\validation.gdb\List_ds_

20200827_03 20200827_03 D:\DS_ML\modely\model_004
Preparation of compo array  D:\DS_ML\modely\model_004\validation.gdb\List_ds_20200827_03_001_validation  is done
(2, 3977, 983)
3909391
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20200827_03  done

20201102_01 20201102_01 D:\DS_ML\modely\model_004
Preparation of compo array  D:\DS_ML\modely\model_004\validation.gdb\List_ds_20201102_01_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20201102_01  done

20201102_04 20201102_04 D:\DS_ML\modely\model_004
Preparation of compo array  D:\DS_ML\modely\model_004\validation.gdb\List_ds_20201102_04_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20201102_04  done

20201102_05 20201102_05 D:\DS_ML\modely\model_004
Preparation of compo array  D:\DS_ML\modely\model_004\validation.gdb\List_ds_

20181219_03 20181219_03 D:\DS_ML\modely\model_005
Preparation of compo array  D:\DS_ML\modely\model_005\validation.gdb\List_ds_20181219_03_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20181219_03  done

20190108_01 20190108_01 D:\DS_ML\modely\model_005
Preparation of compo array  D:\DS_ML\modely\model_005\validation.gdb\List_ds_20190108_01_001_validation  is done
(2, 3958, 983)
3890714
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20190108_01  done

20190108_02 20190108_02 D:\DS_ML\modely\model_005
Preparation of compo array  D:\DS_ML\modely\model_005\validation.gdb\List_ds_20190108_02_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20190108_02  done

20190108_03 20190108_03 D:\DS_ML\modely\model_005
Preparation of compo array  D:\DS_ML\modely\model_005\validation.gdb\List_ds_

20181127_02 20181127_02 D:\DS_ML\modely\model_006
Preparation of compo array  D:\DS_ML\modely\model_006\validation.gdb\List_ds_20181127_02_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20181127_02  done

20181127_03 20181127_03 D:\DS_ML\modely\model_006
Preparation of compo array  D:\DS_ML\modely\model_006\validation.gdb\List_ds_20181127_03_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20181127_03  done

20181129_01 20181129_01 D:\DS_ML\modely\model_006
Preparation of compo array  D:\DS_ML\modely\model_006\validation.gdb\List_ds_20181129_01_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20181129_01  done

20181129_02 20181129_02 D:\DS_ML\modely\model_006
Preparation of compo array  D:\DS_ML\modely\model_006\validation.gdb\List_ds_

20201112_03 20201112_03 D:\DS_ML\modely\model_006
Preparation of compo array  D:\DS_ML\modely\model_006\validation.gdb\List_ds_20201112_03_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20201112_03  done

20201112_04 20201112_04 D:\DS_ML\modely\model_006
Preparation of compo array  D:\DS_ML\modely\model_006\validation.gdb\List_ds_20201112_04_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20201112_04  done

20201112_05 20201112_05 D:\DS_ML\modely\model_006
Preparation of compo array  D:\DS_ML\modely\model_006\validation.gdb\List_ds_20201112_05_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20201112_05  done

20201119_01 20201119_01 D:\DS_ML\modely\model_006
Preparation of compo array  D:\DS_ML\modely\model_006\validation.gdb\List_ds_

...

20200824_01 20200824_01 D:\DS_ML\modely\model_009
Preparation of compo array  D:\DS_ML\modely\model_009\validation.gdb\List_ds_20200824_01_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20200824_01  done

20200824_02 20200824_02 D:\DS_ML\modely\model_009
Preparation of compo array  D:\DS_ML\modely\model_009\validation.gdb\List_ds_20200824_02_001_validation  is done
(2, 3978, 984)
3914352
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20200824_02  done

20200827_02 20200827_02 D:\DS_ML\modely\model_009
Preparation of compo array  D:\DS_ML\modely\model_009\validation.gdb\List_ds_20200827_02_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20200827_02  done

20200827_03 20200827_03 D:\DS_ML\modely\model_009
Preparation of compo array  D:\DS_ML\modely\model_009\validation.gdb\List_ds_

20181214_03 20181214_03 D:\DS_ML\modely\model_010
Preparation of compo array  D:\DS_ML\modely\model_010\validation.gdb\List_ds_20181214_03_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20181214_03  done

20181219_01 20181219_01 D:\DS_ML\modely\model_010
Preparation of compo array  D:\DS_ML\modely\model_010\validation.gdb\List_ds_20181219_01_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20181219_01  done

20181219_02 20181219_02 D:\DS_ML\modely\model_010
Preparation of compo array  D:\DS_ML\modely\model_010\validation.gdb\List_ds_20181219_02_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20181219_02  done

20181219_03 20181219_03 D:\DS_ML\modely\model_010
Preparation of compo array  D:\DS_ML\modely\model_010\validation.gdb\List_ds_

20181124_01 20181124_01 D:\DS_ML\modely\model_011
Preparation of compo array  D:\DS_ML\modely\model_011\validation.gdb\List_ds_20181124_01_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20181124_01  done

20181124_02 20181124_02 D:\DS_ML\modely\model_011
Preparation of compo array  D:\DS_ML\modely\model_011\validation.gdb\List_ds_20181124_02_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20181124_02  done

20181127_01 20181127_01 D:\DS_ML\modely\model_011
Preparation of compo array  D:\DS_ML\modely\model_011\validation.gdb\List_ds_20181127_01_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20181127_01  done

20181127_02 20181127_02 D:\DS_ML\modely\model_011
Preparation of compo array  D:\DS_ML\modely\model_011\validation.gdb\List_ds_

20201105_03 20201105_03 D:\DS_ML\modely\model_011
Preparation of compo array  D:\DS_ML\modely\model_011\validation.gdb\List_ds_20201105_03_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20201105_03  done

20201112_01 20201112_01 D:\DS_ML\modely\model_011
Preparation of compo array  D:\DS_ML\modely\model_011\validation.gdb\List_ds_20201112_01_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20201112_01  done

20201112_02 20201112_02 D:\DS_ML\modely\model_011
Preparation of compo array  D:\DS_ML\modely\model_011\validation.gdb\List_ds_20201112_02_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20201112_02  done

20201112_03 20201112_03 D:\DS_ML\modely\model_011
Preparation of compo array  D:\DS_ML\modely\model_011\validation.gdb\List_ds_

20200716_03 20200716_03 D:\DS_ML\modely\model_012
Preparation of compo array  D:\DS_ML\modely\model_012\validation.gdb\List_ds_20200716_03_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20200716_03  done

20200721_01 20200721_01 D:\DS_ML\modely\model_012
Preparation of compo array  D:\DS_ML\modely\model_012\validation.gdb\List_ds_20200721_01_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20200721_01  done

20200721_02 20200721_02 D:\DS_ML\modely\model_012
Preparation of compo array  D:\DS_ML\modely\model_012\validation.gdb\List_ds_20200721_02_001_validation  is done
(2, 3977, 984)
3913368
10 % , 20 % , 30 % , 40 % , 50 % , 60 % , 70 % , 80 % , 90 % , 
Validation matrix of  20200721_02  done

20200721_03 20200721_03 D:\DS_ML\modely\model_012
Preparation of compo array  D:\DS_ML\modely\model_012\validation.gdb\List_ds_

In [53]:
 Con("Filtered_31\List_Bykovice4_20210510_NoRill_filtered.tif"==0,Con("Filtered_31\List_Bykovice4_20210510_Rill_filtered.tif"==0, Con("Filtered_31\List_Bykovice4_20210510_Sheet_filtered.tif"==0,0,2) ,1),0)

215

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jan  2 12:11:42 2022

From filtered rasters create final judged raster comprising of three classes

@author: adamt
"""

import os
import sys
import arcpy
import numpy as np
import pandas as pd
import csv
from arcpy.sa import *

#%%
def create_RGB_raster(input_NoRill_raster, input_Sheet_raster, input_Rill_raster, output_name):

    ##  loading layers
    gdbPath = r"S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\machine_learning\STRIX_ML.gdb"

    # To allow overwriting outputs change overwrite Output option to True.
    arcpy.env.overwriteOutput = True

    input_NoRill_ras = arcpy.Raster(input_NoRill_raster)
    input_Sheet_ras = arcpy.Raster(input_Sheet_raster)
    input_Rill_ras = arcpy.Raster(input_Rill_raster)
    
    ## cellsize specification
    cellSize = input_NoRill_ras.meanCellWidth
    Cell_Size = str(cellSize)+" "+str(cellSize)

    ## additional values specification
    lowerLeft = arcpy.Point(input_NoRill_ras.extent.XMin,input_NoRill_ras.extent.YMin)

    ## export to numpy array
    input_NoRill_array = arcpy.RasterToNumPyArray(input_NoRill_ras,nodata_to_value=255)
    input_Sheet_array = arcpy.RasterToNumPyArray(input_Sheet_ras,nodata_to_value=255)
    input_Rill_array = arcpy.RasterToNumPyArray(input_Rill_ras,nodata_to_value=255)

    ## array dimensions
    dimensions = input_NoRill_array.shape
    height, width = dimensions[0], dimensions[1]
    
    print(input_NoRill_array.shape, input_Sheet_array.shape, input_Rill_array.shape)

    print("Preparation of compo array ", input_NoRill_raster," is done")
    print(input_NoRill_array.shape)
        
    result_red_array = np.zeros((height, width))
    result_green_array = np.zeros((height, width))
    result_blue_array = np.zeros((height, width))
    executed_array = np.zeros((height, width))
    
    # statistical data
    statistics = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    check = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    # seting the basic parameters
    max_x = width-1
    max_y = height-1
    # max_x = 1000
    # max_y = 1000
    ra = width * height
    
    NoRill, Sheet, Rill = 0, 1, 2

    # zatridovaci strom

    side = 0
    i,j = 0, 0
    a = 0
    c = 0
    percento = 0
    switch = 0
    C = 0

#     result_red_array = 2.54 * input_NoRill_array + 1
#     result_green_array = 2.54 * input_Sheet_array + 1
#     result_blue_array = 2.54 * input_Rill_array + 1
    
    result_red_array = 2.55 * input_NoRill_array
    result_green_array = 2.55 * input_Sheet_array
    result_blue_array = 2.55 * input_Rill_array

    print(statistics)
    
    result_red_array = result_red_array.astype(int)
    result_green_array = result_green_array.astype(int)
    result_blue_array = result_blue_array.astype(int)
    
    print(result_red_array.shape)
    print(result_green_array.shape)
    print(result_blue_array.shape)
            
#     result_red_array = np.concatenate((result_red_array, result_green_array), axis=2)
#     result_red_array = np.concatenate((result_red_array, result_blue_array), axis=2)
    result_array = np.stack((result_red_array, result_green_array, result_blue_array), axis=0)
    
    resultRaster = arcpy.NumPyArrayToRaster(result_array, lowerLeft, cellSize, value_to_nodata=255)
#     processRaster = arcpy.NumPyArrayToRaster(process_array, lowerLeft, cellSize, value_to_nodata=255)

    resultRaster.save(output_name)
#     process_name = output_name.replace('filtered', 'processed')
#     processRaster.save(process_name)

#     projection = "PROJCS['S-JTSK_Krovak_East_North',GEOGCS['GCS_S_JTSK',DATUM['D_S_JTSK',SPHEROID['Bessel_1841',6377397.155,299.1528128]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Krovak'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Pseudo_Standard_Parallel_1',78.5],PARAMETER['Scale_Factor',0.9999],PARAMETER['Azimuth',30.28813975277778],PARAMETER['Longitude_Of_Center',24.83333333333333],PARAMETER['Latitude_Of_Center',49.5],PARAMETER['X_Scale',-1.0],PARAMETER['Y_Scale',1.0],PARAMETER['XY_Plane_Rotation',90.0],UNIT['Meter',1.0]]"

#     arcpy.management.DefineProjection(resultRaster, projection)

    print('Analysis of array ', input_NoRill_raster," is done")
    

In [6]:
import os

# storage_location = r"E:\DS_ML\model_001"
log_list = ['20181112_01', '20181112_02', '20181112_03', '20181115_02', '20181115_03', '20181123_02', '20181123_03', '20181124_01', '20181124_02', '20181127_01', '20181127_02', '20181127_03', '20181129_01', '20181129_02', '20181129_03', '20181203_01', '20181203_02', '20181203_03', '20181203_04', '20181206_01',
            '20181206_02', '20181206_03', '20181211_01', '20181211_02', '20181211_03', '20181211_04', '20181211_05', '20181211_06', '20181214_01', '20181214_02', '20181214_03', '20181219_01', '20181219_02', '20181219_03', '20190108_01', '20190108_02', '20190108_03', '20200708_01', '20200708_02', '20200716_01',
            '20200716_03', '20200721_01', '20200721_02', '20200721_03', '20200727_01', '20200727_02', '20200727_03', '20200729_01', '20200729_02', '20200729_03', '20200803_01', '20200803_03', '20200810_01', '20200824_01', '20200824_02', '20200827_02', '20200827_03', '20201102_01', '20201102_04',
            '20201102_05', '20201102_06', '20201105_01', '20201105_02', '20201105_03', '20201112_01', '20201112_02', '20201112_03', '20201112_04', '20201112_05', '20201119_01', '20201119_02', '20201119_03', '20201119_04', '20201119_05', '20201126_01', '20201126_02', '20201126_03', '20201126_04', '20201203_06']

# log_list = []

model_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
# model_list = ['01']

for model in model_list:
    model_name = "model_0" + model
    storage_location = os.path.join(r"S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely", model_name)

    for item in log_list:
    #     loc = item.split('_')
    #     loc = loc[0] + "_" + loc[1]
        loc = item
        print(item, loc)

        input_NoRill_raster = os.path.join(storage_location, "result_rasters", "List_ds_" + loc + "_noRill_avg.tif")
        input_Sheet_raster = os.path.join(storage_location, "result_rasters", "List_ds_" + loc + "_sheet_avg.tif")
        input_Rill_raster = os.path.join(storage_location, "result_rasters", "List_ds_" + loc + "_rill_avg.tif")

        output_name = os.path.join(storage_location, "RGB_rasters_1", "List_ds_" + loc + "_RGB.tif")

        create_RGB_raster(input_NoRill_raster, input_Sheet_raster, input_Rill_raster, output_name)

print('done')

20181112_01 20181112_01
(159, 39) (159, 39) (159, 39)
Preparation of compo array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_001\result_rasters\List_ds_20181112_01_noRill_avg.tif  is done
(159, 39)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(159, 39)
(159, 39)
(159, 39)
Analysis of array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_001\result_rasters\List_ds_20181112_01_noRill_avg.tif  is done
20181112_02 20181112_02
(159, 39) (159, 39) (159, 39)
Preparation of compo array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_001\result_rasters\List_ds_20181112_02_noRill_avg.tif  is done
(159, 39)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(159, 39)
(159, 39)
(159, 39)
Analysis of array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_001\result_rasters\List_ds_20181112_02_noRill_a

Preparation of compo array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_001\result_rasters\List_ds_20181219_02_noRill_avg.tif  is done
(159, 39)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(159, 39)
(159, 39)
(159, 39)
Analysis of array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_001\result_rasters\List_ds_20181219_02_noRill_avg.tif  is done
20181219_03 20181219_03
(159, 39) (159, 39) (159, 39)
Preparation of compo array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_001\result_rasters\List_ds_20181219_03_noRill_avg.tif  is done
(159, 39)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(159, 39)
(159, 39)
(159, 39)
Analysis of array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_001\result_rasters\List_ds_20181219_03_noRill_avg.tif  is done
20190108_01 20190108_01
(158, 39) (158

Preparation of compo array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_001\result_rasters\List_ds_20201112_01_noRill_avg.tif  is done
(159, 39)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(159, 39)
(159, 39)
(159, 39)
Analysis of array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_001\result_rasters\List_ds_20201112_01_noRill_avg.tif  is done
20201112_02 20201112_02
(159, 39) (159, 39) (159, 39)
Preparation of compo array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_001\result_rasters\List_ds_20201112_02_noRill_avg.tif  is done
(159, 39)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(159, 39)
(159, 39)
(159, 39)
Analysis of array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_001\result_rasters\List_ds_20201112_02_noRill_avg.tif  is done
20201112_03 20201112_03
(159, 39) (159

Preparation of compo array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_002\result_rasters\List_ds_20181203_03_noRill_avg.tif  is done
(159, 39)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(159, 39)
(159, 39)
(159, 39)
Analysis of array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_002\result_rasters\List_ds_20181203_03_noRill_avg.tif  is done
20181203_04 20181203_04
(159, 39) (159, 39) (159, 39)
Preparation of compo array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_002\result_rasters\List_ds_20181203_04_noRill_avg.tif  is done
(159, 39)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(159, 39)
(159, 39)
(159, 39)
Analysis of array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_002\result_rasters\List_ds_20181203_04_noRill_avg.tif  is done
20181206_01 20181206_01
(159, 39) (159

Preparation of compo array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_002\result_rasters\List_ds_20200729_03_noRill_avg.tif  is done
(159, 39)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(159, 39)
(159, 39)
(159, 39)
Analysis of array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_002\result_rasters\List_ds_20200729_03_noRill_avg.tif  is done
20200803_01 20200803_01
(159, 39) (159, 39) (159, 39)
Preparation of compo array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_002\result_rasters\List_ds_20200803_01_noRill_avg.tif  is done
(159, 39)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(159, 39)
(159, 39)
(159, 39)
Analysis of array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_002\result_rasters\List_ds_20200803_01_noRill_avg.tif  is done
20200803_03 20200803_03
(159, 39) (159

...

Analysis of array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_011\result_rasters\List_ds_20181129_01_noRill_avg.tif  is done
20181129_02 20181129_02
(80, 20) (80, 20) (80, 20)
Preparation of compo array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_011\result_rasters\List_ds_20181129_02_noRill_avg.tif  is done
(80, 20)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(80, 20)
(80, 20)
(80, 20)
Analysis of array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_011\result_rasters\List_ds_20181129_02_noRill_avg.tif  is done
20181129_03 20181129_03
(80, 20) (80, 20) (80, 20)
Preparation of compo array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_011\result_rasters\List_ds_20181129_03_noRill_avg.tif  is done
(80, 20)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(80, 20)
(80, 20)
(80, 20

Preparation of compo array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_011\result_rasters\List_ds_20200727_02_noRill_avg.tif  is done
(80, 20)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(80, 20)
(80, 20)
(80, 20)
Analysis of array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_011\result_rasters\List_ds_20200727_02_noRill_avg.tif  is done
20200727_03 20200727_03
(80, 20) (80, 20) (80, 20)
Preparation of compo array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_011\result_rasters\List_ds_20200727_03_noRill_avg.tif  is done
(80, 20)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(80, 20)
(80, 20)
(80, 20)
Analysis of array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_011\result_rasters\List_ds_20200727_03_noRill_avg.tif  is done
20200729_01 20200729_01
(80, 20) (80, 20) (80, 20

Analysis of array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_011\result_rasters\List_ds_20201126_04_noRill_avg.tif  is done
20201203_06 20201203_06
(80, 20) (80, 20) (80, 20)
Preparation of compo array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_011\result_rasters\List_ds_20201203_06_noRill_avg.tif  is done
(80, 20)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(80, 20)
(80, 20)
(80, 20)
Analysis of array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_011\result_rasters\List_ds_20201203_06_noRill_avg.tif  is done
20181112_01 20181112_01
(80, 20) (80, 20) (80, 20)
Preparation of compo array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_012\result_rasters\List_ds_20181112_01_noRill_avg.tif  is done
(80, 20)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(80, 20)
(80, 20)
(80, 20

Preparation of compo array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_012\result_rasters\List_ds_20181219_01_noRill_avg.tif  is done
(80, 20)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(80, 20)
(80, 20)
(80, 20)
Analysis of array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_012\result_rasters\List_ds_20181219_01_noRill_avg.tif  is done
20181219_02 20181219_02
(80, 20) (80, 20) (80, 20)
Preparation of compo array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_012\result_rasters\List_ds_20181219_02_noRill_avg.tif  is done
(80, 20)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(80, 20)
(80, 20)
(80, 20)
Analysis of array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_012\result_rasters\List_ds_20181219_02_noRill_avg.tif  is done
20181219_03 20181219_03
(80, 20) (80, 20) (80, 20

Analysis of array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_012\result_rasters\List_ds_20201105_03_noRill_avg.tif  is done
20201112_01 20201112_01
(80, 20) (80, 20) (80, 20)
Preparation of compo array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_012\result_rasters\List_ds_20201112_01_noRill_avg.tif  is done
(80, 20)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(80, 20)
(80, 20)
(80, 20)
Analysis of array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_012\result_rasters\List_ds_20201112_01_noRill_avg.tif  is done
20201112_02 20201112_02
(80, 20) (80, 20) (80, 20)
Preparation of compo array  S:\Private\_PROJEKTY\2017_Strix_svahy\Tejkl_ML\modely\model_012\result_rasters\List_ds_20201112_02_noRill_avg.tif  is done
(80, 20)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
(80, 20)
(80, 20)
(80, 20